In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['NE']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('norepinephrine', 5467),
 ('neutrophil elastase', 703),
 ('nuclear envelope', 516),
 ('neuroendocrine', 297),
 ('noradrenaline', 104),
 ('noradrenergic', 78),
 ('necrotic enteritis', 68),
 ('nanoemulsion', 60),
 ('nuclear extract', 53),
 ('negative emotionality', 38),
 ('nocturnal enuresis', 34),
 ('neonatal encephalopathy', 27),
 ('neuroectoderm', 25),
 ('nephropathia epidemica', 19),
 ('neuroepithelial', 16),
 ('nephrine', 14),
 ('nor epinephrine', 13),
 ('net energy', 12),
 ('neural ectoderm', 10),
 ('neuroepithelium', 9),
 ('norepine phrine', 7),
 ('noladin ether', 6),
 ('neutrophil', 6),
 ('nutrition education', 5),
 ('network', 5),
 ('novel environment', 5),
 ('neighborhood environment', 5),
 ('norepinephrinergic', 5),
 ('norepineprine', 5),
 ('nash equilibrium', 4),
 ('neostriatum', 4),
 ('neutralizing epitope', 4),
 ('norephinephrine', 4),
 ('norepinephine', 4),
 ('norepinepherine', 4),
 ('norethisterone', 4),
 ('northeast', 4),
 ('neuro epithelial like', 4),
 ('norepinephrin

In [6]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [7]:
names

{'CHEBI:CHEBI:18357': '(R)-noradrenaline',
 'GO:GO:0005635': 'nuclear envelope',
 'CHEBI:CHEBI:33569': 'noradrenaline',
 'MESH:D004751': 'Enteritis',
 'MESH:D053206': 'Nocturnal Enuresis',
 'DOID:DOID:0050201': 'nephropathia epidemica',
 'CHEBI:CHEBI:28918': '(R)-adrenaline',
 'MESH:D004475': 'Ectoderm',
 'MESH:C424442': 'noladin ether',
 'MESH:D009504': 'Neutrophils',
 'MESH:D004493': 'Education',
 'MESH:D004777': 'Environment',
 'MESH:D017072': 'Neostriatum',
 'CHEBI:CHEBI:53000': 'epitope',
 'CHEBI:CHEBI:7627': 'norethisterone',
 'HGNC:3309': 'ELANE',
 'MESH:D017599': 'Neuroectodermal Tumors',
 'MESH:D044504': 'Enterocolitis, Neutropenic',
 'EFO:0000487': 'exposure',
 'CHEBI:CHEBI:33250': 'atom',
 'MESH:D004947': 'Esophagus'}

In [8]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-11-10 04:55:29] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [9]:
result = [grounding_map, names, pos_labels]

In [10]:
result

[{'nanoemulsion': 'ungrounded',
  'nash equilibria': 'ungrounded',
  'nash equilibrium': 'ungrounded',
  'ne': 'ungrounded',
  'necrotic enteritis': 'ungrounded',
  'negative emotionality': 'ungrounded',
  'neighborhood environment': 'ungrounded',
  'neonatal encephalopathy': 'ungrounded',
  'neostriatum': 'MESH:D017072',
  'nephrine': 'ungrounded',
  'nephropathia epidemica': 'ungrounded',
  'net energy': 'ungrounded',
  'network': 'ungrounded',
  'neural ectoderm': 'ungrounded',
  'neuro epithelial like': 'ungrounded',
  'neuroectoderm': 'ungrounded',
  'neuroendocrine': 'ungrounded',
  'neuroepithelial': 'ungrounded',
  'neuroepithelioma': 'MESH:D017599',
  'neuroepithelium': 'ungrounded',
  'neutralizing epitope': 'ungrounded',
  'neutropenic enterocolitis': 'MESH:D044504',
  'neutrophil': 'HGNC:3309',
  'neutrophil elastase': 'HGNC:3309',
  'nicotine equivalent': 'ungrounded',
  'nocturnal enuresis': 'ungrounded',
  'noise exposed': 'ungrounded',
  'noise exposure': 'ungrounded',


In [11]:
grounding_map, names, pos_labels = [{'nanoemulsion': 'ungrounded',
  'nash equilibria': 'ungrounded',
  'nash equilibrium': 'ungrounded',
  'ne': 'ungrounded',
  'necrotic enteritis': 'ungrounded',
  'negative emotionality': 'ungrounded',
  'neighborhood environment': 'ungrounded',
  'neonatal encephalopathy': 'ungrounded',
  'neostriatum': 'MESH:D017072',
  'nephrine': 'ungrounded',
  'nephropathia epidemica': 'ungrounded',
  'net energy': 'ungrounded',
  'network': 'ungrounded',
  'neural ectoderm': 'ungrounded',
  'neuro epithelial like': 'ungrounded',
  'neuroectoderm': 'ungrounded',
  'neuroendocrine': 'ungrounded',
  'neuroepithelial': 'ungrounded',
  'neuroepithelioma': 'MESH:D017599',
  'neuroepithelium': 'ungrounded',
  'neutralizing epitope': 'ungrounded',
  'neutropenic enterocolitis': 'MESH:D044504',
  'neutrophil': 'HGNC:3309',
  'neutrophil elastase': 'HGNC:3309',
  'nicotine equivalent': 'ungrounded',
  'nocturnal enuresis': 'ungrounded',
  'noise exposed': 'ungrounded',
  'noise exposure': 'ungrounded',
  'noladin ether': 'MESH:C424442',
  'non e box': 'ungrounded',
  'non essential': 'ungrounded',
  'non exercise': 'ungrounded',
  'nor epinephrine': 'CHEBI:CHEBI:28918',
  'noradrenaline': 'CHEBI:CHEBI:28918',
  'noradrenergic': 'CHEBI:CHEBI:28918',
  'norephinephrine': 'CHEBI:CHEBI:28918',
  'norepine phrine': 'CHEBI:CHEBI:28918',
  'norepinepherine': 'CHEBI:CHEBI:28918',
  'norepinephine': 'CHEBI:CHEBI:28918',
  'norepinephrine': 'CHEBI:CHEBI:28918',
  'norepinephrine bitartrate': 'CHEBI:CHEBI:28918',
  'norepinephrinergic': 'CHEBI:CHEBI:28918',
  'norepineprine': 'CHEBI:CHEBI:28918',
  'norethisterone': 'CHEBI:CHEBI:7627',
  'normal esophageal': 'ungrounded',
  'normal esophagus': 'ungrounded',
  'northeast': 'ungrounded',
  'novel environment': 'ungrounded',
  'nuclear': 'ungrounded',
  'nuclear envelope': 'GO:GO:0005635',
  'nuclear extract': 'ungrounded',
  'nutrition education': 'ungrounded',
  'nutrition elements': 'ungrounded'},
 {'MESH:D017072': 'Neostriatum',
  'MESH:D017599': 'Neuroectodermal Tumors',
  'MESH:D044504': 'Enterocolitis, Neutropenic',
  'HGNC:3309': 'ELANE',
  'MESH:C424442': 'noladin ether',
  'CHEBI:CHEBI:28918': '(R)-adrenaline',
  'CHEBI:CHEBI:7627': 'norethisterone',
  'GO:GO:0005635': 'nuclear envelope'},
 ['CHEBI:CHEBI:28918', 'GO:GO:0005635', 'HGNC:3309']]

In [12]:
excluded_longforms = ['ne']

In [13]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [14]:
additional_entities = {}

In [15]:
unambiguous_agent_texts = {}

In [16]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [18]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [19]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [20]:
intersection1

[('HGNC:10967', 'HGNC:10967', 98)]

In [21]:
intersection2

[('GO:GO:0001837', 'HGNC:10967', 0),
 ('ungrounded', 'HGNC:10967', 0),
 ('HGNC:10967', 'HGNC:10967', 0),
 ('MESH:D055032', 'HGNC:10967', 0)]

In [22]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [23]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [17]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-10 05:01:37] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-10 05:03:31] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9832163602852344 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [18]:
classifier.stats

{'label_distribution': {'CHEBI:CHEBI:28918': 4671,
  'GO:GO:0005635': 390,
  'HGNC:3309': 482,
  'ungrounded': 518,
  'CHEBI:CHEBI:7627': 2,
  'MESH:C424442': 5,
  'MESH:D017599': 2,
  'MESH:D044504': 2,
  'MESH:D017072': 1},
 'f1': {'mean': 0.983216, 'std': 0.002514},
 'precision': {'mean': 0.980138, 'std': 0.00209},
 'recall': {'mean': 0.986649, 'std': 0.002992},
 'CHEBI:CHEBI:28918': {'f1': {'mean': 0.987463, 'std': 0.001448},
  'pr': {'mean': 0.994862, 'std': 0.002184},
  'rc': {'mean': 0.980181, 'std': 0.002726}},
 'GO:GO:0005635': {'f1': {'mean': 0.970302, 'std': 0.01872},
  'pr': {'mean': 0.966667, 'std': 0.026399},
  'rc': {'mean': 0.974307, 'std': 0.018332}},
 'CHEBI:CHEBI:7627': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'ungrounded': {'f1': {'mean': 0.913805, 'std': 0.010019},
  'pr': {'mean': 0.889937, 'std': 0.010117},
  'rc': {'mean': 0.939127, 'std': 0.015189}},
 'MESH:D044504': {'f1': {'mean': 0.0, 'std': 0.

In [19]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [20]:
disamb.dump(model_name, results_path)

In [21]:
print(disamb.info())

Disambiguation model for NE

Produces the disambiguations:
	(R)-adrenaline*	CHEBI:CHEBI:28918
	ELANE*	HGNC:3309
	Enterocolitis, Neutropenic	MESH:D044504
	Neostriatum	MESH:D017072
	Neuroectodermal Tumors	MESH:D017599
	noladin ether	MESH:C424442
	norethisterone	CHEBI:CHEBI:7627
	nuclear envelope*	GO:GO:0005635

Class level metrics:
--------------------
Grounding                 	Count	F1     
            (R)-adrenaline*	4671	0.98746
                Ungrounded	 518	 0.9138
                     ELANE*	 482	0.95244
          nuclear envelope*	 390	 0.9703
             noladin ether	   5	    0.4
            norethisterone	   2	    0.0
    Neuroectodermal Tumors	   2	    0.0
Enterocolitis, Neutropenic	   2	    0.0
               Neostriatum	   1	    0.0

Weighted Metrics:
-----------------
	F1 score:	0.98322
	Precision:	0.98014
	Recall:		0.98665

* Positive labels
See Docstring for explanation



In [22]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1